In [ ]:
import pandas as pd

In [ ]:
df=pd.read_excel('complete.xlsx', index_col=0)
CURRENT_YEAR = 2022

In [ ]:
df = df.groupby(['SubjectName', 'Place', 'Time', 'Teacher'])['Date'].agg(Dates = lambda x: x).reset_index()
df['Dates'] = df['Dates'].apply(lambda x: parse_dates(x))
df = df.explode('Dates')
df[['DateStart', 'DateEnd', 'RepeatTime', 'RepeatType']] = pd.DataFrame(df.Dates.tolist(), index=df.index)
df.head()

In [ ]:
import datetime as dt


def parse_dates(dates):
    if type(dates) == pd.core.arrays.datetimes.DatetimeArray:
        # dates = dates.reset_index(drop=True)
        return find_pattern(dates)
    else:
        return [(dates, dates, 1, '')]


def find_pattern(dates):
    output_data = []
    date_start = dates[0]
    date_end = dates[0]
    repeat_time = 1
    repeat_type = 'weekly'
    for date in dates[1:]:
        if (date_end + dt.timedelta(days=7) == date and repeat_type == 'weekly') or (
                date_end + dt.timedelta(days=14) == date and repeat_type == 'biweekly'):
            date_end = date
            repeat_time += 1
        else:
            if date_end + dt.timedelta(days=7) == date:
                if repeat_time == 1:
                    repeat_time += 1
                    date_end = date
                    repeat_type = 'weekly'
                    continue
                output_data.append((date_start, date_end, repeat_time, repeat_type))
                date_start, date_end, repeat_time, repeat_type = date, date, 1, 'weekly'
                continue
            if date_end + dt.timedelta(days=14) == date:
                if repeat_time == 1:
                    repeat_time += 1
                    date_end = date
                    repeat_type = 'biweekly'
                    continue
                output_data.append((date_start, date_end, repeat_time, repeat_type))
                date_start, date_end, repeat_time, repeat_type = date, date, 1, 'biweekly'
                continue
            if date_end == date:
                continue
            output_data.append((date_start, date_end, repeat_time, repeat_type))
            date_start, date_end, repeat_time = date, date, 1
    output_data.append((date_start, date_end, repeat_time, repeat_type))
    return output_data


def parse_time(time_str):
    def get_datetime_format(time_str: str):
        return pd.to_datetime(time_str, format='%H:%M').time()

    return list(map(get_datetime_format, time_str.split('–')))

In [ ]:
df[['TimeStart', 'TimeEnd']] = pd.DataFrame(df['Time'].apply(parse_time).tolist(), index=df.index)
df = df.drop(['Dates', 'Time'], axis=1)

In [ ]:
df = df[df['SubjectName'].str.match('((^(?!Траектория).*)(^(?!Русский язык).*))|(Траектория 3 .*)')]

In [ ]:
df

In [ ]:
df.to_csv('spring.csv')

In [ ]:
from icalendar import Calendar, Event
import pytz

In [ ]:
# Убран функционал подгрупп!!
class Subject:
    params = ['TimeStart', 'TimeEnd', 'DateStart', 'DateEnd', 'SubjectName', 'Place', 'Teacher', 'RepeatTime',
              'RepeatType']  #, 'SubGroup'
    tz = pytz.timezone('Europe/Moscow')

    def __init__(self, TimeStart, TimeEnd, DateStart, DateEnd, SubjectName, Place, Teacher, RepeatTime=1, RepeatType='',
                 SubGroup=None):
        self.time_start = TimeStart
        self.time_end = TimeEnd
        self.date_start = DateStart
        self.date_end = DateEnd
        self.subject_name = SubjectName if not SubGroup else SubjectName + '#' + f'{SubGroup}'
        self.place = Place
        self.teacher = Teacher
        self.repeat_time = RepeatTime
        self.repeat_type = RepeatType
        self.sub_group = SubGroup

    @property
    def date_time_start_event(self):
        return self.get_date_time(self.date_start, self.time_start)

    @property
    def date_time_end_event(self):
        return self.get_date_time(self.date_start, self.time_end)

    @classmethod
    def get_date_time(cls, date, time):
        return dt.datetime.combine(date, time, cls.tz)

    @classmethod
    def from_series(cls, series: pd.Series):
        subject_params = [series[param] for param in cls.params]
        return cls(*subject_params)

    def __repr__(self):
        return f"<Subject time start: {self.time_start} date start: {self.date_start} with {self.repeat_time} repeat>"

    def to_event(self) -> Event:
        e = Event()
        e.add('DTSTART', self.date_time_start_event)
        e.add('DTEND', self.date_time_end_event)
        e.add('SUMMARY', self.subject_name)
        e.add('DESCRIPTION', f"Преподователь: {self.teacher}")
        e.add('LOCATION', self.place)
        e.add('LAST-MODIFIED', dt.datetime.now(self.tz))
        if self.repeat_type:
            e.add('RRULE',
                  {'freq': 'weekly', 'count': self.repeat_time, 'interval': 1 if self.repeat_type == 'weekly' else 2})
        return e

In [ ]:
cal = Calendar()
cal.add('prodid', '-//Karpovich//Karpovich Alex//EN')

In [ ]:
for idx, item in df.iterrows():
    s = Subject.from_series(item)
    e = s.to_event()
    cal.add_component(e)

In [ ]:
with open('example_spring.ics', 'wb') as f:
    data = cal.to_ical()
    f.write(data)